In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
parent_dir = os.path.dirname(os.getcwd())
path = parent_dir 
save_path = parent_dir + '/'
from optimization import *
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
color_palette = ['#332288',  '#117733',  '#88CCEE', '#DDCC77', '#CC6677', '#AA4499', '#882255']

## Figure 2: Annual V2G Costs and Scatterplot

In [ ]:
period_string = '2024-06-03_to_2025-06-01'

#read cluster data
num_clusters = 15
cluster_labels = np.array(pd.read_csv('Data/Cluster_Results_Adaptive/cluster_labels_'+str(num_clusters)+'.csv', header=None).iloc[:,0])
cluster_centers = pd.read_csv('Data/Cluster_Results_Adaptive/cluster_centers_'+str(num_clusters)+'.csv', header=None)
vinids = np.array(pd.read_csv('Data/vinids.csv').iloc[:,1])

#make a dataframe of vinids and cluster labels
vinids_df = pd.DataFrame({'vinid': vinids, 'cluster': cluster_labels})
label_list = np.unique(vinids_df['cluster'])

In [ ]:
def replace_nan_with_previous_value(cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere):
    '''Replace any nan weeks with the previous week's value and reshape the matrices to Jan-Jan format.'''
    for vin in np.arange(cost_mat_controlled.shape[0]):
        for week in np.arange(cost_mat_controlled.shape[1]):
            if np.isnan(cost_mat_controlled[vin, week]):
                if week == 0:
                    cost_mat_controlled[vin, week] = 0
                else:
                    cost_mat_controlled[vin, week] = cost_mat_controlled[vin, week-1]
            if np.isnan(cost_mat_uncontrolled[vin, week]):
                if week == 0:
                    cost_mat_uncontrolled[vin, week] = 0
                else:
                    cost_mat_uncontrolled[vin, week] = cost_mat_uncontrolled[vin, week-1]
            if np.isnan(cost_mat_v2g_home[vin, week]):
                if week == 0:
                    cost_mat_v2g_home[vin, week] = 0
                else:
                    cost_mat_v2g_home[vin, week] = cost_mat_v2g_home[vin, week-1]
            if np.isnan(cost_mat_v2g_everywhere[vin, week]):
                if week == 0:
                    cost_mat_v2g_everywhere[vin, week] = 0
                else:
                    cost_mat_v2g_everywhere[vin, week] = cost_mat_v2g_everywhere[vin, week-1]


    #find rows that sum up to zero from controlled and remove them from cost matrices
    zero_idx = np.all(cost_mat_controlled == 0, axis=1)
    cost_mat_controlled = cost_mat_controlled[~zero_idx]
    cost_mat_uncontrolled = cost_mat_uncontrolled[~zero_idx]
    cost_mat_v2g_home = cost_mat_v2g_home[~zero_idx]
    cost_mat_v2g_everywhere = cost_mat_v2g_everywhere[~zero_idx]

    #order the matrices so data is Jan-Jan
    cost_mat_uncontrolled_jan = np.zeros((cost_mat_uncontrolled.shape[0], 52))
    cost_mat_controlled_jan = np.zeros((cost_mat_controlled.shape[0], 52))
    cost_mat_v2g_home_jan = np.zeros((cost_mat_v2g_home.shape[0], 52))
    cost_mat_v2g_everywhere_jan = np.zeros((cost_mat_v2g_everywhere.shape[0], 52)) 

    num_weeks = 52
    year_splice = 22
    year_splice_2 = num_weeks-year_splice
    cost_mat_uncontrolled_jan[:, 0:year_splice] = cost_mat_uncontrolled[:, year_splice_2:num_weeks]
    cost_mat_uncontrolled_jan[:, year_splice:num_weeks] = cost_mat_uncontrolled[:, 0:year_splice_2]
    cost_mat_controlled_jan[:, 0:year_splice] = cost_mat_controlled[:, year_splice_2:num_weeks] 
    cost_mat_controlled_jan[:, year_splice:num_weeks] = cost_mat_controlled[:, 0:year_splice_2]
    cost_mat_v2g_home_jan[:, 0:year_splice] = cost_mat_v2g_home[:, year_splice_2:num_weeks]
    cost_mat_v2g_home_jan[:, year_splice:num_weeks] = cost_mat_v2g_home[:, 0:year_splice_2]
    cost_mat_v2g_everywhere_jan[:, 0:year_splice] = cost_mat_v2g_everywhere[:, year_splice_2:num_weeks]
    cost_mat_v2g_everywhere_jan[:, year_splice:num_weeks] = cost_mat_v2g_everywhere[:, 0:year_splice_2]

    return cost_mat_uncontrolled_jan, cost_mat_controlled_jan, cost_mat_v2g_home_jan, cost_mat_v2g_everywhere_jan


In [ ]:
def load_results(results_path, period_string):
    '''Load the cost results into matrices from the specified path and period string.'''
    vin_length = 749
    cost_mat_uncontrolled = []
    cost_mat_controlled = []
    cost_mat_v2g_home = []
    cost_mat_v2g_everywhere = []

    for vin in np.arange(1,vin_length):
        #skip the vins that do not have data 
        try:
            cost_mat_uncontrolled.append( pd.read_csv(results_path + 'cost_uncontrolled_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_controlled.append( pd.read_csv(results_path + 'cost_no_v2g_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_v2g_home.append( pd.read_csv(results_path + 'cost_v2g_home_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_v2g_everywhere.append(pd.read_csv(results_path + 'cost_v2g_everywhere_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])

        except:
            pass

    cost_mat_uncontrolled = np.array(cost_mat_uncontrolled)
    cost_mat_controlled = np.array(cost_mat_controlled)
    cost_mat_v2g_home = np.array(cost_mat_v2g_home)
    cost_mat_v2g_everywhere = np.array(cost_mat_v2g_everywhere)
    return cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere

In [ ]:
def get_box_data(vinids_df, results_path, period_string):    
    '''Load cost results to be broken down into clusters'''
    vinids_df = pd.DataFrame({'vinid': vinids, 'cluster': cluster_labels})
    vin_length = 749
    cost_mat_uncontrolled = []
    cost_mat_controlled = []
    cost_mat_v2g_home = []
    cost_mat_v2g_everywhere = []
    vin_not_found_idx = []

    for vin in np.arange(1,vin_length):
        try:
            cost_mat_uncontrolled.append( pd.read_csv(results_path + 'cost_uncontrolled_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_controlled.append( pd.read_csv(results_path + 'cost_no_v2g_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_v2g_home.append( pd.read_csv(results_path + 'cost_v2g_home_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_v2g_everywhere.append(pd.read_csv(results_path + 'cost_v2g_everywhere_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])

        except:
            vin_not_found_idx.append(np.where(vinids==vin)[0])

    cost_mat_uncontrolled = np.array(cost_mat_uncontrolled)
    cost_mat_controlled = np.array(cost_mat_controlled)
    cost_mat_v2g_home = np.array(cost_mat_v2g_home)
    cost_mat_v2g_everywhere = np.array(cost_mat_v2g_everywhere)

    for remove in vin_not_found_idx:
        try:
            vinids_df = vinids_df.drop(vinids_df[vinids_df['vinid'] == vinids[remove[0]]].index)
        except:
            pass
    vinids_df = vinids_df.reset_index(drop=True)

    #replace nan with previous value
    for vin in np.arange(cost_mat_controlled.shape[0]):
        for week in np.arange(cost_mat_controlled.shape[1]):
            if np.isnan(cost_mat_controlled[vin, week]):
                if week == 0:
                    cost_mat_controlled[vin, week] = 0
                else:
                    cost_mat_controlled[vin, week] = cost_mat_controlled[vin, week-1]
            if np.isnan(cost_mat_uncontrolled[vin, week]):
                if week == 0:
                    cost_mat_uncontrolled[vin, week] = 0
                else:
                    cost_mat_uncontrolled[vin, week] = cost_mat_uncontrolled[vin, week-1]
            if np.isnan(cost_mat_v2g_home[vin, week]):
                if week == 0:
                    cost_mat_v2g_home[vin, week] = 0
                else:
                    cost_mat_v2g_home[vin, week] = cost_mat_v2g_home[vin, week-1]
            if np.isnan(cost_mat_v2g_everywhere[vin, week]):
                if week == 0:
                    cost_mat_v2g_everywhere[vin, week] = 0
                else:
                    cost_mat_v2g_everywhere[vin, week] = cost_mat_v2g_everywhere[vin, week-1]

    #delete rows that sum up to zero from controlled
    zero_idx = np.all(cost_mat_controlled == 0, axis=1)
    cost_mat_controlled = cost_mat_controlled[~zero_idx]
    cost_mat_uncontrolled = cost_mat_uncontrolled[~zero_idx]
    cost_mat_v2g_home = cost_mat_v2g_home[~zero_idx]
    cost_mat_v2g_everywhere = cost_mat_v2g_everywhere[~zero_idx]

    # drop the rows where zero_idx is True from vinids_df
    vinids_df = vinids_df[~zero_idx]
    vinids_df = vinids_df.reset_index(drop=True)
    return vinids_df, cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere


In [ ]:
def plot_costs(ax, cost_mat_uncontrolled, cost_mat_v2g_home, cost_mat_v2g_everywhere, circuit):

    cost_dict = {
        'V2G Home': np.mean(np.cumsum(cost_mat_uncontrolled, axis=1) - np.cumsum(cost_mat_v2g_home, axis=1), axis=0)[-1],
        'V2G Everywhere': np.mean(np.cumsum(cost_mat_uncontrolled, axis=1) - np.cumsum(cost_mat_v2g_everywhere, axis=1), axis=0)[-1]
    }

    cost_std_dict = {
        'V2G Home': np.std(np.cumsum(cost_mat_uncontrolled, axis=1) - np.cumsum(cost_mat_v2g_home, axis=1), axis=0)[-1],
        'V2G Everywhere': np.std(np.cumsum(cost_mat_uncontrolled, axis=1) - np.cumsum(cost_mat_v2g_everywhere, axis=1), axis=0)[-1]
    }

    x = int(circuit[-1]) + int(circuit[-1])*0.18  # the label locations
    width = 0.35  # the width of the bars


    multiplier = 0
    for attribute, measurement in cost_dict.items():
        offset = width * multiplier 
        rects = ax.bar(x + offset + .6, measurement, width, label=attribute, color = color_palette[multiplier+2])
        ax.bar_label(rects, labels = ['$'+str(int(measurement))], padding=cost_std_dict[attribute]*0.068, fontsize=11)

        multiplier += 1
        if attribute in cost_std_dict:
            ax.errorbar(x + offset +0.6, measurement, yerr=cost_std_dict[attribute], fmt='none', color='black', capsize=5)

    min=0
    max=5500
    ax.set_ylim(min, max)
    ax.set_yticklabels([str(int(x/1000)) for x in np.arange(min, max+1, 1000)], fontsize=15)   
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.set_ylabel('Annual Charging Savings with V2G [$1000]', fontsize=15)
    if circuit == 'cir_1':
        ax.legend(fontsize=13, loc='upper right')

    #remove xticks and xticklabels
    ax.set_xticks(np.arange(1, 5)*1.18+0.6+width/2)
    ax.set_xticklabels(['Circuit 1', 'Circuit 2', 'Circuit 3', 'Circuit 4'])

    return


In [ ]:
def plot_heatmap(ax, heatmap_data):
    heatmap_data = np.array(heatmap_data)
    #reduce the size of heatmap data by binning the data 
    heatmap_reduced = []
    for price in np.arange(heatmap_data[:,0].min(), heatmap_data[:,0].max(), 0.001):
        mask = (heatmap_data[:,0] >= price) & (heatmap_data[:,0] < price + 0.001)
        if np.any(mask):
            for area_under_curve in np.arange(heatmap_data[:,1].min(), heatmap_data[:,1].max(), 0.01):
                mask2 = mask & (heatmap_data[:,1] >= area_under_curve) & (heatmap_data[:,1] < area_under_curve + 0.01)
                if np.any(mask2):
                    heatmap_reduced.append([price, area_under_curve, np.mean(heatmap_data[mask2,2])])

    heatmap_reduced = np.array(heatmap_reduced)
    scatter = ax.scatter(heatmap_reduced[:,0], heatmap_reduced[:,1], c=heatmap_reduced[:,2], cmap='winter', s=70)
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.set_xlabel('Average Dynamic Price [$]', fontsize=15)
    ax.set_ylabel('Average Charging Availability', fontsize=15)
    
    return scatter

In [ ]:
#read in price data from each circuit (EV2A) and then calculate the average price that week and then save it
for i, circuit_ID in enumerate(['012041131', '022011162', '252051104', '182191101']):
    circuit_data = pd.read_csv(f'Hourly_Prices/HourlyFlexPricing_PGE_EV2AP_'+circuit_ID + '.csv')
    #assign a week number for each of the 52 weeks
    circuit_data['week'] = (circuit_data.index // 168) + 1
    circuit_data.groupby('week').Price.mean().to_csv(f'Hourly_Prices/circuit_{i+1}_mean_price.csv', index=False)


In [ ]:
aging = 'batt_aging_0'
period_string = '2024-06-03_to_2025-06-01'

fig, ax = plt.subplots(1, 2, figsize=(19, 6))
ax = ax.flatten()
plt.subplots_adjust(wspace=0.15)

cluster_info_df = pd.read_csv('Data/Cluster_Results_Adaptive/cluster_info_'+str(num_clusters)+'.csv')
heatmap_data = []
for i, circuit in enumerate(['cir_1', 'cir_2', 'cir_3', 'cir_4']):
    print('Circuit ' + str(circuit)[-1])
    circuit_prices = np.array(pd.read_csv(f'Hourly_Prices/circuit_{i+1}_mean_price.csv'))
    #load results
    results_path = 'Results/'+ circuit+ '/' + aging + '/' + 'elrp_1_'
    cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere = load_results(results_path, period_string)
    cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere = replace_nan_with_previous_value(cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere)
    num_periods = cost_mat_uncontrolled.shape[1]
    plot_costs(ax[0], cost_mat_uncontrolled, cost_mat_v2g_home, cost_mat_v2g_everywhere,  circuit)
    print(str(np.round(np.mean(np.cumsum( cost_mat_uncontrolled, axis=1), axis=0)[-1],2)) + ' & ' + str(np.round(np.mean(np.cumsum( cost_mat_controlled, axis=1), axis=0)[-1], 2)) + ' & ' + str(np.round(np.mean(np.cumsum( cost_mat_v2g_home, axis=1), axis=0)[-1], 2)) + ' & ' + str(np.round(np.mean(np.cumsum( cost_mat_v2g_everywhere, axis=1), axis=0)[-1], 2)) )

    #assign data to heatmap array
    vinids_df, cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere = get_box_data(vinids_df, results_path, period_string)
    
    for c in range(num_clusters):

        num_periods = cost_mat_uncontrolled.shape[1]
        c_label = cluster_info_df['Orig Cluster Label'].values[c]
        chg_avail = np.trapz(cluster_centers.iloc[c, :].values) / 60 / 24
        
        for week_price in cost_mat_v2g_home[vinids_df.cluster == c_label, :]  :
            for week in range(52):
                mean_price = circuit_prices[week] / 100
                heatmap_data.append([mean_price[0], chg_avail, week_price[week]])

#plot scatterplot and set the colorbar args
scatter = plot_heatmap(ax[1],heatmap_data)
cbar = plt.colorbar(scatter, ax=ax[1], pad=0.1)
cbar.ax.tick_params(labelsize=13)
cbar.set_label(label='Weekly Charging Cost with V2G [$]', size=14)
cbar.ax.yaxis.set_ticks_position('left')

#annotate a) and b)
ax[0].text(-0.07, 1.02, 'a.', transform 
            = ax[0].transAxes, fontsize=18)
ax[1].text(-0.15, 1.02, 'b.', transform 
            = ax[1].transAxes, fontsize=18)

plt.savefig(save_path + '2_heatmap_costs.pdf', bbox_inches='tight')